In [0]:
%%sh
aws s3 ls s3://b2c-prod-data-pipeline-unified-dna/unified/dna.dna_universal_company_parent.v1/dimension/  --recursive --human --summarize

In [0]:
%%sh
PGPASSWORD='2mHdFW6%#REu' psql -h internal-aa-prod-plproxy-internal-4-329644124.us-east-1.elb.amazonaws.com -p 7432 -U app_bdp_usage_qa -d dailyest << EOF
SELECT company_id, pub_id, company_type, date_range, date 
 FROM plproxy.execute_select_nestloop(\$proxy\$ 
  SELECT company_id, pub_id, company_type, date_range, date
    FROM pp.company_publisher_map_0
    where pub_id=20200000142711 
    LIMIT 100;
     \$proxy\$) tbl (company_id BIGINT, pub_id BIGINT, company_type SMALLINT, date_range DATERANGE, date DATE)  LIMIT 100 ;
EOF

In [0]:
%%sh
# and date::timestamp <@ '[2014-07-08, infinity)'::tsrange
PGPASSWORD='2mHdFW6%#REu' psql -h internal-aa-prod-plproxy-internal-4-329644124.us-east-1.elb.amazonaws.com -p 7432 -U app_bdp_usage_qa -d dailyest << EOF
SELECT * 
 FROM plproxy.execute_select_nestloop(\$proxy\$ 
  SELECT * 
    FROM pp.pub_app_daily_associate_0 
    LIMIT 10;
     \$proxy\$) tbl (pub_id BIGINT, category_id INTEGER, app_id BIGINT, date DATE)  LIMIT 10 ;
EOF

In [0]:
%%sh
PGPASSWORD='2mHdFW6%#REu' psql -h 10.2.25.198  -U app_bdp_usage_qa -d dna -p 6432 << EOF 
set search_path=public;
-- \d public.dna_universal_company_parent;
SELECT s.universal_id uni_id_s, t.universal_id uni_id_t, s.parent_id, s.start_date, s.end_date, t.company_id, t.company_name 
FROM public.dna_universal_company_parent s, public.dna_universal_company_mapping t 
WHERE s.parent_id = t.universal_id AND 
NOT s.disabled AND 
s.confirmed AND 
NOT t.disabled AND 
t.confirmed 
LIMIT 2;

-- SELECT * 
-- FROM public.dna_universal_company_parent s 
-- LIMIT 1;
EOF

In [0]:
%%sh
PGPASSWORD='2mHdFW6%#REu' psql -h 10.2.25.198  -U app_bdp_usage_qa -d dna -p 6432 << EOF 
set search_path=public;
-- \d public.dna_universal_company_mapping;
SELECT * FROM public.dna_universal_company_mapping where universal_id=1000200000003635 LIMIT 100;
EOF

In [0]:

df = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-dna/unified/dna.dna_universal_company_parent.v1/dimension/")
df.show(2)
df.createOrReplaceTempView('dna_universal_company_parent')

In [0]:

from applications.db_check_v1.common.db_check_utils import query, pg_settings, query_df
sql = 'SELECT * FROM public.dna_universal_company_parent'
db_df = query_df(pg_settings('dna'), sql)


In [0]:

df2 = spark.sql(
    "SELECT * "
    "FROM dna_universal_company_parent "
    "WHERE id=4687"
    )
df2.show(1)

In [0]:

uni_df = df.toPandas()
print uni_df.T

In [0]:

uni_df.drop(['dimension_id', '_partition_col', '_identifier', '_update_date', 'comments'], axis=1, inplace=True)
print uni_df.T

In [0]:

# str(db_df['created_time'][0])[:10]
db_df['created_time'] = [str(i)[:10] for i in db_df['created_time']]
db_df['last_updated_time'] = [str(i)[:10] for i in db_df['created_time']]
db_df.drop('comments', axis=1, inplace=True)
print db_df

In [0]:

diff_df = uni_df.merge(db_df, how='outer', on=['id'], indicator=True)
if diff_df.loc[diff_df['_merge'] != 'both'].empty:
    print "SUCCESS"
else:
    print diff_df.loc[diff_df['_merge'] != 'both']

In [0]:

diff_df.loc[diff_df['_merge'] == 'both']

In [0]:

print df.count()
print df.distinct().count()

In [0]:

from bdce.common.utils import update_application_code
update_application_code(spark, role="BDP-PROD-APP-INT-QA", application_name="zidong-application-autopipeline")

# reload dependencies from temp
spark.sparkContext.addPyFile("/tmp/zeppelin_application_code/libs/python/dependencies.zip")
# spark.sparkContext.addPyFile("/home/hadoop/bdp/application/libs/python/dependencies.zip")
import aaplproxy


In [0]:
%%sh
# ls /home/hadoop/bdp/application/aadatapipelinecore/core/
ls /tmp/zeppelin_application_code/


In [0]:
%%sh
PGPASSWORD='2mHdFW6%#REu' psql -h 10.2.25.198  -U app_bdp_usage_qa -d dna -p 6432 << EOF 
set search_path=public;
\d public.dna_universal_app_mapping;
--select * from public.dna_universal_app_mapping limit 3;
select count(*) from public.dna_universal_app_mapping where app_id in (
SELECT t1.product_id_dna
FROM 
(SELECT *, int.product_id as product_id_int, dna.product_id as product_id_dna
FROM public.in_app_dna_info_mapping int RIGHT JOIN dna_genre_id_product_mapping dna ON int.product_id = dna.product_id
WHERE int.product_id is NULL) t1
WHERE not (t1.product_id_dna > 30000000000000 and t1.product_id_dna < 40000000000000)) and confirmed='f';


-- select * from public.dna_universal_app_mapping where app_id='1356497382' limit 3;
EOF

In [0]:
%%sh
PGPASSWORD='2mHdFW6%#REu' psql -h 10.2.25.198  -U app_bdp_usage_qa -d dna -p 6432 << EOF 
set search_path=public;
\d public.dna_universal_company_mapping;
-- select * from public.dna_universal_company_mapping where company_id='1473244723' limit 3;
--select * from  public.dna_universal_company_mapping limit 10;
EOF

In [0]:
%%sh
PGPASSWORD='2mHdFW6%#REu' psql -h 10.2.25.198  -U app_bdp_usage_qa -d dna -p 6432 << EOF 
set search_path=public;
\d public.dna_universal_company_parent;
select * from public.dna_universal_company_parent limit 3;
EOF

In [0]:
%%sh
PGPASSWORD='2mHdFW6%#REu' psql -h 10.2.25.198  -U app_bdp_usage_qa -d dna -p 6432 << EOF 
set search_path=public;
-- explain (analyze True, costs True) 
-- SELECT *
-- FROM 
-- (SELECT *, int.product_id as product_id_int, dna.product_id as product_id_dna
-- FROM public.in_app_dna_info_mapping int RIGHT JOIN dna_genre_id_product_mapping dna ON int.product_id = dna.product_id
-- WHERE int.product_id is NULL) t1
-- WHERE not (t1.product_id_dna > 30000000000000 and t1.product_id_dna < 40000000000000) limit 10;
-- 
-- SELECT count(*)
-- FROM public.in_app_dna_info_mapping int LEFT JOIN dna_genre_id_product_mapping dna ON int.product_id = dna.product_id
-- WHERE dna.product_id is NULL limit 3;
-- select count(*) from public.in_app_dna_info_mapping limit 3;
-- \d public.in_app_dna_info_mapping;
select * from public.in_app_dna_info_mapping limit 3;
select * from public.in_app_dna_info_mapping where company_id='1000200000118216' limit 3;
select company_id from public.in_app_dna_info_mapping group by company_id limit 3;
EOF

In [0]:
%%sh
PGPASSWORD='2mHdFW6%#REu' psql -h 10.2.25.198  -U app_bdp_usage_qa -d dna -p 6432 << EOF 
set search_path=public;
-- SELECT count(*)
-- FROM dna_genre_id_product_mapping

-- select count(*) from dna_genre_id_product_mapping limit 3;
-- \d dna_genre_id_product_mapping;
select * from dna_genre_id_product_mapping limit 3;
EOF

In [0]:
%%sh
PGPASSWORD='wZw8cfBuuklIskVG' psql -h 10.2.10.254  -U citus_bdp_prod_app_int_qa -d aa_store_db -p 5432 << EOF 
set search_path=store;
select count(*) from store_dna_map_dimension_v1;
select * from store_dna_map_dimension_v1 limit 4;
select * from store_dna_map_dimension_v1 where unified_app_id is NULL and genre_id is not NULL limit 4;
select * from (
    select unified_app_id, app_id 
    from store_dna_map_dimension_v1 
    WHERE unified_app_id is not NULL 
    GROUP BY unified_app_id, app_id) t1 
ORDER BY unified_app_id
LIMIT 4;
EOF

In [0]:
%%sh
aws s3 ls s3://b2c-prod-data-pipeline-unified-store-paid/unified/

In [0]:


from conf.settings import PG_DNA_NAME, PG_DNA_ACCESS_ID, PG_DNA_HOSTS, PG_DNA_SECRET_KEY
from applications.db_check_v1.common.db_check_utils import query_df
from pyspark.sql.functions import explode

dna_dsn = (
    "dbname='{db}' user='{user}' password='{password}' "
    "host='{host}' port='{port}'".format(
        db=PG_DNA_NAME,
        user=PG_DNA_ACCESS_ID,
        host=PG_DNA_HOSTS[0][0],
        password=PG_DNA_SECRET_KEY,
        port=PG_DNA_HOSTS[0][1]
    )
)
# mapping_df_unified = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-dna/unified/dna.genre_id_product_mapping.v1/dimension/").toPandas()
sql = 'SELECT * FROM dna_genre_id_product_mapping limit 3'
mapping_df_raw = query_df(dna_dsn, sql)
mapping_df_raw.info(verbose=True)
print mapping_df_raw
# spark_df = spark.createDataFrame(mapping_df_raw)
# spark_df.show(3)
# mapping_df_raw = mapping_df_raw.explode('genre_id')
# exploded_spark_df = spark_df.select(explode('genre_id').alias('genre_id'))
# exploded_spark_df.show(3)
# genre_count = exploded_spark_df.distinct().count()

In [0]:

from pyspark.sql import types as T
from pyspark.sql import functions as F
from aadatapipelinecore.core.urn import Urn
from aadatapipelinecore.core.pipeline import type_
from applications.common.parser import SqlParser
from applications.common.executor import SqlExecutor
from applications.auto_pipeline.transform import _view
import aaplproxy

class DryRunSqlExecutor(SqlExecutor):
    def _verify_tasks(self):
        pass
def run(spark, raw_data, sql_text, dry_run=True):
    urn = Urn(namespace=raw_data["namespace"])
    source_data_list = raw_data.pop("source")
    raw_data.update(raw_data.pop("options"))
    _view(spark, sql_text, None, source_data_list)
    context = raw_data
    tasks = SqlParser(spark, sql_text, context).parse()
    if dry_run:
        sql_executor = DryRunSqlExecutor
    else:
        sql_executor = SqlExecutor
    sql_executor(urn, spark, tasks, type_.EventType.TRANSFORM, context).run()
sql_text ="""
WITH dna_mapping AS (
    SELECT
        db_table(
            engine="PG",
            database="DNA",
            sql="select count(*) from dna_genre_id_product_mapping LIMIT 3"
        )
);

WITH in_mapping AS (
    SELECT
        db_table(
            engine='PG',
            database='DNA',
            sql="SELECT count(*) FROM in_app_dna_info_mapping"
        )
);
"""
test_date='2020-03-24'
namespace = "aa.store.market-size.v1"
ingest_msg = {
    "namespace": "aa.store.market-size.v1",
    "job_type": "routine",
    "options":{},
    "source": [
        {
            "name":"rank_raw",
            "data_encoding": "parquet",
            "compression": "gzip",
            "path": ["s3://b2c-prod-dca-store-estimates/store_estv2/APP_ESTIMATES_FINAL/version=2.0.0/range_type=DAY/date=2020-03-28/"],
        }
    ]
}
run(spark, ingest_msg, sql_text)
spark.sql("select * from dna_mapping LIMIT 3").show(3)
spark.sql("select * from in_mapping LIMIT 3").show(3)

In [0]:


from pyspark.sql import types as T
from pyspark.sql import functions as F

from aadatapipelinecore.core.urn import Urn
from aadatapipelinecore.core.pipeline import type_
from applications.common.parser import SqlParser
from applications.common.executor import SqlExecutor
from applications.auto_pipeline.transform import _view
import aaplproxy

class DryRunSqlExecutor(SqlExecutor):
    def _verify_tasks(self):
        pass
def run(spark, raw_data, sql_text, dry_run=True):
    urn = Urn(namespace=raw_data["namespace"])
    source_data_list = raw_data.pop("source")
    raw_data.update(raw_data.pop("options"))
    _view(spark, sql_text, None, source_data_list)
    context = raw_data
    tasks = SqlParser(spark, sql_text, context).parse()
    if dry_run:
        sql_executor = DryRunSqlExecutor
    else:
        sql_executor = SqlExecutor
    sql_executor(urn, spark, tasks, type_.EventType.TRANSFORM, context).run()
sql_text = """
WITH temp_table AS (
select * from store_unified LIMIT 5
);
"""

# store_unified , rank_unified
namespace = "aa.store.market-size.v1"
ingest_msg = {
    "namespace": "aa.store.market-size.v1",
    "job_type": "routine",
    "options":{},
    "source": [
        {
            "name":"rank_raw",
            "data_encoding": "parquet",
            "compression": "gzip",
            "path": ["s3://b2c-prod-dca-store-estimates/store_estv2/APP_ESTIMATES_FINAL/version=2.0.0/range_type=DAY/date=2020-03-28/"],
        },        {
            "data_encoding": "parquet",
            "compression": "gzip",
          "name":"store_unified",
            "path": ["s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est.v1/fact/granularity=daily/date=2020-03-28"],
        },        {
            "data_encoding": "parquet",
            "compression": "gzip",
          "name":"category_mapping_deminsion_service",
            "path": ["s3://b2c-prod-data-pipeline-qa/aa.store/store_cateogry_mapping"],
        }, 
        {
            "data_encoding": "parquet",
            "compression": "gzip",
          "name":"rank_unified",
            "path": ["s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est-category-rank.v1/fact/granularity=daily/date=2020-03-28"],
        },
        {
            "data_encoding": "csv",
            "compression": "gzip",
            "name":"ios_country_mapping",
            "data_schema": [
                            {"name":"store_id","type":"int","nullable": False},
                            {"name":"country_code","type":"string","nullable": False},
                            {"name":"country_name","type":"string","nullable": False}
                            ],
             "csv_options": {
           'header': True,
           'sep': '\t',
           'quote': '',
           'encoding': 'utf-8',
           'escape': ''
           },

            "path": ["s3://b2c-prod-dca-store-estimates/store_back/dimension/IOS_COUNTRY_MAPPING"],
        },   
        {
            "data_encoding": "csv",
            "compression": "gzip",
            "name":"android_country_mapping",
            "data_schema": [
                            {"name":"store_id","type":"int","nullable": False},
                            {"name":"country_code","type":"string","nullable": False},
                            {"name":"country_name","type":"string","nullable": False}
                            ],
             "csv_options": {
           'header': True,
           'sep': '\t',
           'quote': '',
           'encoding': 'utf-8',
           'escape': ''
           },

            "path": ["s3://b2c-prod-dca-store-estimates/store_back/dimension/ANDROID_COUNTRY_MAPPING"],
        }
    ]
}
run(spark, ingest_msg, sql_text)
spark.sql("select * from temp_table ").show()